In [ ]:
import pandas as pd
import requests
# from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import urllib.parse
import re
import numpy as np
import seaborn as sns
import ast
from wordcloud import WordCloud, STOPWORDS
import sklearn
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.decomposition import PCA
import ast

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_parquet('/Users/jiazhengli/Desktop/project_root/clinical_trials_interventional.parquet')
print(df.shape)
df.head()

# df = pd.read_parquet('/Users/jiazhengli/Desktop/project_root/clinical_trials_interventional.parquet', engine='pyarrow', columns=None).head(10000)
# print(df.shape)
# df.head()

In [ ]:
df['protocolSection.statusModule.overallStatus'].value_counts()

In [ ]:
selected_statuses = ["COMPLETED", "TERMINATED", "SUSPENDED", "WITHDRAWN"]

df = df[df['protocolSection.statusModule.overallStatus'].isin(selected_statuses)].reset_index(drop=True)
# df.shape

In [ ]:
df.columns = df.columns.str.split('.', n=1).str[1].str.replace('.', '_', regex=False)

In [ ]:
columns_to_keep = ['identificationModule_nctId', 'statusModule_overallStatus', 'designModule_phases', 'identificationModule_organization_class', 'oversightModule_oversightHasDmc', 'oversightModule_isFdaRegulatedDrug', 'oversightModule_isFdaRegulatedDevice',
'designModule_designInfo_allocation', 'designModule_designInfo_interventionModel', 'designModule_designInfo_primaryPurpose', 'designModule_designInfo_maskingInfo_masking', 'eligibilityModule_healthyVolunteers', 'eligibilityModule_sex', 'moreInfoModule_certainAgreement_piSponsorEmployee',
'moreInfoModule_certainAgreement_restrictiveAgreement', 'statusModule_completionDateStruct_date',
'statusModule_startDateStruct_date', 'eligibilityModule_eligibilityCriteria', 'contactsLocationsModule_locations',
'conditionBrowseModule_ancestors', 'sponsorCollaboratorsModule_collaborators', 'interventionBrowseModule_meshes',
'descriptionModule_briefSummary', 'descriptionModule_detailedDescription'
]

df = df[columns_to_keep]
print(df.shape)
df.head()

Encode Phase

In [ ]:
# Rename values in the 'Phase' column

# def fix_phases(phases):
#     if pd.isna(phases):  # Handle NaN values
#         return ['NA']
#     if isinstance(phases, str):  # Convert string to a Python list
#         try:
#             phases = ast.literal_eval(phases)  # Safely parse the string
#         except (ValueError, SyntaxError):  # Handle any malformed strings
#             return ['NA']
#     if isinstance(phases, list):  # Process the list
#         return ['PHASE1' if phase == 'EARLY_PHASE1' else phase for phase in phases]
#     return ['NA']  # Default to ['NA'] for unexpected cases

# df['designModule_phases'] = df['designModule_phases'].apply(fix_phases)
def fix_phases(phases):
    if isinstance(phases, float) and pd.isna(phases):  # Handle NaN values
        return ['NA']

    if isinstance(phases, np.ndarray):  # Convert NumPy arrays to Python lists
        phases = phases.tolist()

    if isinstance(phases, list):  # Process lists correctly
        return ['PHASE1' if phase == 'EARLY_PHASE1' else phase for phase in phases]

    return ['NA']  # Default case

# Apply function with `.loc` to avoid `SettingWithCopyWarning`
df.loc[:, 'designModule_phases'] = df['designModule_phases']

df['designModule_phases'].value_counts()

In [ ]:
# Ensure the column contains only valid lists
df['designModule_phases'] = df['designModule_phases'].apply(
    lambda x: x.tolist() if isinstance(x, np.ndarray) else x  # Convert NumPy arrays to lists
)

# Create dummy variables for each phase
valid_phases = ['PHASE1', 'PHASE2', 'PHASE3', 'PHASE4', 'NA']

for phase in valid_phases:
    df[phase] = df['designModule_phases'].apply(lambda x: int(phase in x) if isinstance(x, list) else 0)

# Display the dummy columns
print(df[valid_phases].head())

In [ ]:
df = df.drop(columns=['NA','designModule_phases'])

In [ ]:
# def create_dummies(df, column_name):
#     """
#     Creates dummy variables for a categorical column, handles missing values, and drops the original column and 'NA' dummy.

#     Parameters:
#         df (pd.DataFrame): The DataFrame containing the column to process.
#         column_name (str): The name of the column to transform.

#     Returns:
#         pd.DataFrame: The DataFrame with dummy variables added and the original column removed.
#     """
#     # Check for missing values and fill with 'NA'
#     df[column_name] = df[column_name].fillna('NA')

#     # Create dummy variables
#     dummies = pd.get_dummies(df[column_name], prefix="", prefix_sep="")

#     # Drop the 'NA' column if it exists
#     if 'NA' in dummies.columns:
#         dummies = dummies.drop(columns=['NA'])

#     # Drop the original column
#     df = df.drop(columns=[column_name])

#     # Concatenate the dummies with the original DataFrame
#     df = pd.concat([df, dummies], axis=1)

#     return df
def create_dummies(df, column_name):
    """
    Creates dummy variables for a categorical column, handles missing values, and drops the original column and 'NA' dummy.
    """
    df = df.copy()  # Avoid modifying the original DataFrame
    df[column_name] = df[column_name].fillna('NA')
    dummies = pd.get_dummies(df[column_name], prefix="", prefix_sep="")

    if 'NA' in dummies.columns:
        dummies = dummies.drop(columns='NA')

    df = pd.concat([df.drop(columns=column_name), dummies], axis=1)
    return df


Encode organization class

In [ ]:
df = create_dummies(df, 'identificationModule_organization_class')
df = df.rename(columns={
    'FED': 'organization_class_FED',
    'INDIV': 'organization_class_INDIV',
    'INDUSTRY': 'organization_class_INDUSTRY',
    'NETWORK': 'organization_class_NETWORK',
    'NIH': 'organization_class_NIH',
    'OTHER': 'organization_class_OTHER',
    'OTHER_GOV': 'organization_class_OTHER_GOV',
    'UNKNOWN': 'organization_class_UNKNOWN'
})

In [ ]:
df = create_dummies(df, 'oversightModule_oversightHasDmc')
df = df.rename(columns={
    'False': 'oversightHasDmc_no',
    'True': 'oversightHasDmc_yes'
})

In [ ]:
df = create_dummies(df, 'oversightModule_isFdaRegulatedDrug')
df = df.rename(columns={
    'False': 'FdaRegulatedDrug_no',
    'True': 'FdaRegulatedDrug_yes'
})

In [ ]:
df = create_dummies(df, 'oversightModule_isFdaRegulatedDevice')
df = df.rename(columns={
    'False': 'FdaRegulatedDevice_no',
    'True': 'FdaRegulatedDevice_yes'
})

In [ ]:
df = create_dummies(df, 'designModule_designInfo_allocation')
df = df.rename(columns={
    'NON_RANDOMIZED': 'designInfo_allocation_NON_RANDOMIZED',
    'RANDOMIZED': 'designInfo_allocation_RANDOMIZED'
})

In [ ]:
df = create_dummies(df, 'designModule_designInfo_interventionModel')
df = df.rename(columns={
    'CROSSOVER': 'interventionModel_CROSSOVER',
    'FACTORIAL': 'interventionModel_FACTORIAL',
    'PARALLEL': 'interventionModel_PARALLEL',
    'SEQUENTIAL': 'interventionModel_SEQUENTIAL',
    'SINGLE_GROUP': 'interventionModel_SINGLE_GROUP'
})

In [ ]:
df = create_dummies(df, 'designModule_designInfo_primaryPurpose')
df = df.rename(columns={
    'BASIC_SCIENCE': 'primaryPurpose_BASIC_SCIENCE',
    'DEVICE_FEASIBILITY': 'primaryPurpose_DEVICE_FEASIBILITY',
    'DIAGNOSTIC': 'primaryPurpose_DIAGNOSTIC',
    'ECT': 'primaryPurpose_ECT',
    'HEALTH_SERVICES_RESEARCH': 'primaryPurpose_HEALTH_SERVICES_RESEARCH',
    'OTHER': 'primaryPurpose_OTHER',
    'PREVENTION': 'primaryPurpose_PREVENTION',
    'SCREENING': 'primaryPurpose_SCREENING',
    'SUPPORTIVE_CARE': 'primaryPurpose_SUPPORTIVE_CARE',
    'TREATMENT': 'primaryPurpose_TREATMENT'
})

In [ ]:
df = create_dummies(df, 'designModule_designInfo_maskingInfo_masking')
df = df.rename(columns={
    'DOUBLE': 'masking_DOUBLE',
    'NONE': 'masking_NONE',
    'QUADRUPLE': 'masking_QUADRUPLE',
    'SINGLE': 'masking_SINGLE',
    'TRIPLE': 'masking_TRIPLE'
})

In [ ]:
df = create_dummies(df, 'eligibilityModule_healthyVolunteers')
df = df.rename(columns={
    'False': 'healthyVolunteers_no',
    'True': 'healthyVolunteers_yes'
})

In [ ]:
df = create_dummies(df, 'eligibilityModule_sex')
df = df.rename(columns={
    'ALL': 'sex_all',
    'FEMALE': 'sex_female',
    'MALE': 'sex_male'
})

In [ ]:
df = create_dummies(df, 'moreInfoModule_certainAgreement_piSponsorEmployee')
df = df.rename(columns={
    'False': 'piSponsorEmployee_no',
    'True': 'piSponsorEmployee_yes'
})

In [ ]:
df = create_dummies(df, 'moreInfoModule_certainAgreement_restrictiveAgreement')
df = df.rename(columns={
    'False': 'restrictiveAgreement_no',
    'True': 'restrictiveAgreement_yes'
})

In [ ]:
def fill_incomplete_date(date_str):
    if pd.isna(date_str):
        return date_str  # Leave NaN as is
    if len(date_str) == 7:  # Format like '2004-10'
        return date_str + '-15'  # Add '-15' for the middle of the month
    return date_str

# Clean and fill incomplete dates
df['statusModule_startDateStruct_date'] = df['statusModule_startDateStruct_date'].apply(fill_incomplete_date)
df['statusModule_completionDateStruct_date'] = df['statusModule_completionDateStruct_date'].apply(fill_incomplete_date)

# Convert to datetime, leaving NaN values as is
df['statusModule_startDateStruct_date'] = pd.to_datetime(df['statusModule_startDateStruct_date'], errors='coerce')
df['statusModule_completionDateStruct_date'] = pd.to_datetime(df['statusModule_completionDateStruct_date'], errors='coerce')

# Calculate duration in years for non-missing dates
df['duration_of_trial'] = (
    (df['statusModule_completionDateStruct_date'] - df['statusModule_startDateStruct_date']).dt.days / 365
)

# Group by 'statusModule_overallStatus' to calculate the average duration for each status
average_durations = (
    df[df['duration_of_trial'].notna()]
    .groupby('statusModule_overallStatus')['duration_of_trial']
    .mean()
)
# Fill missing durations with the average duration based on 'statusModule_overallStatus'
def fill_missing_duration(row):
    if pd.isna(row['duration_of_trial']):
        return average_durations.get(row['statusModule_overallStatus'], None)  # Default to None if no status match
    return row['duration_of_trial']

df['duration_of_trial'] = df.apply(fill_missing_duration, axis=1)

df = df.drop(columns=['statusModule_startDateStruct_date', 'statusModule_completionDateStruct_date'])


In [ ]:
df['duration_of_trial'] = df['duration_of_trial'].fillna(df['duration_of_trial'].mean())
print("Missing values in 'duration_of_trial':", df['duration_of_trial'].isna().sum())

In [ ]:
def convert_age(value):
    if pd.isna(value):  # Handle missing values
        return None
    elif "Weeks" in value:
        return round(int(value.split()[0]) / 52, 2)  # Convert weeks to years
    elif "Years" in value:
        return int(value.split()[0])  # Extract numerical part
    return None  # If unrecognized format

# Apply conversion to both columns
df['eligibilityModule_minimumAge'] = df['eligibilityModule_minimumAge'].apply(convert_age)
df['eligibilityModule_maximumAge'] = df['eligibilityModule_maximumAge'].apply(convert_age)

# Replace missing values
df['eligibilityModule_minimumAge'].fillna(df['eligibilityModule_minimumAge'].min(), inplace=True)
df['eligibilityModule_maximumAge'].fillna(df['eligibilityModule_maximumAge'].max(), inplace=True)

df['eligibilityModule_maximumAge'] = df['eligibilityModule_maximumAge'].apply(lambda x: 100 if x > 100 else x)
print(df['eligibilityModule_minimumAge'][:10])
print(df['eligibilityModule_maximumAge'][:10])

In [ ]:
df['designModule_enrollmentInfo_count'].fillna(df['designModule_enrollmentInfo_count'].median(), inplace=True)

In [ ]:
rare_categories = ['AMBIG', 'UNKNOWN', 'INDIV']

# Replace rare categories with "OTHER_SMALL"
df['sponsorCollaboratorsModule_leadSponsor_class'] = df['sponsorCollaboratorsModule_leadSponsor_class'].replace(rare_categories, 'OTHER_SMALL')

# Create dummies while adding a prefix
df = pd.get_dummies(df, columns=['sponsorCollaboratorsModule_leadSponsor_class'], prefix='leadsponsor')

In [ ]:
def count_criteria(criteria_text):
    inclusion_count = 0
    exclusion_count = 0

    # Ensure the input is a valid string and not NaN
    if isinstance(criteria_text, str) and 'Inclusion Criteria' in criteria_text and 'Exclusion Criteria' in criteria_text:
        # Split into inclusion and exclusion sections
        try:
            inclusion_section = criteria_text.split('Inclusion Criteria:')[1].split('Exclusion Criteria:')[0]
            exclusion_section = criteria_text.split('Exclusion Criteria:')[1]

            # Count bullet points (each * represents a new criterion)
            inclusion_count = inclusion_section.count('*')
            exclusion_count = exclusion_section.count('*')
        except IndexError:
            pass  # In case there's any issue with splitting or formatting

    return inclusion_count, exclusion_count

# Apply the function to the eligibility column
df[['inclusion_count', 'exclusion_count']] = df['eligibilityModule_eligibilityCriteria'].apply(lambda x: pd.Series(count_criteria(x)))

In [ ]:
df = df.drop(['eligibilityModule_eligibilityCriteria'], axis = 1)
df[['inclusion_count', 'exclusion_count']]

In [ ]:
def count_facilities(locations):
    if isinstance(locations, float) and pd.isna(locations):  # Handle NaN values
        return 0

    if isinstance(locations, np.ndarray):  # Convert NumPy arrays to lists
        locations = locations.tolist()

    if isinstance(locations, list):  # Directly count list items
        return len(locations)

    if isinstance(locations, str):  # Convert string representation of a list
        try:
            parsed_locations = ast.literal_eval(locations)
            if isinstance(parsed_locations, list):
                return len(parsed_locations)
        except (ValueError, SyntaxError):
            return 0  # If parsing fails, return 0

    return 0  # Default case for unexpected formats

# Apply function
df['facility_count'] = df['contactsLocationsModule_locations'].apply(count_facilities)

In [ ]:
df = df.drop(['contactsLocationsModule_locations'], axis = 1)
df['facility_count']

In [ ]:
disease_type = [
    'Infections', 'Neoplasms', 'Musculoskeletal Diseases', 'Digestive System Diseases',
    'Stomatognathic Diseases', 'Respiratory Tract Diseases', 'Otorhinolaryngologic Diseases',
    'Nervous System Diseases', 'Eye Diseases', 'Urogenital Diseases', 'Cardiovascular Diseases',
    'Hemic and Lymphatic Diseases', 'Congenital, Hereditary, and Neonatal Diseases and Abnormalities',
    'Skin and Connective Tissue Diseases', 'Nutritional and Metabolic Diseases',
    'Endocrine System Diseases', 'Immune System Diseases', 'Disorders of Environmental Origin',
    'Animal Diseases', 'Pathological Conditions, Signs and Symptoms', 'Occupational Diseases',
    'Chemically-Induced Disorders', 'Wounds and Injuries']

disease_type_set = set(disease_type)
disease_type_lower = set(dt.lower() for dt in disease_type)

# 2. Define parsing functions
# def parse_ancestors(entry):
#     if isinstance(entry, str):
#         try:
#             return ast.literal_eval(entry)
#         except (ValueError, SyntaxError) as e:
#             print(f"Parsing error: {e} for entry: {entry[:50]}...")
#             return []
#     else:
#         return []

# def extract_disease_terms(ancestors, disease_type_set):
#     if not isinstance(ancestors, list):
#         return []
#     return [entry.get('term') for entry in ancestors if entry.get('term') in disease_type_set]

# def extract_unique_sorted_terms(ancestors, disease_type_set):
#     if not isinstance(ancestors, list):
#         return []
#     filtered_terms = {entry.get('term') for entry in ancestors if entry.get('term') in disease_type_set}
#     return sorted(filtered_terms)

# def extract_disease_terms_case_insensitive(ancestors, disease_type_lower_set):
#     if not isinstance(ancestors, list):
#         return []
#     return [entry.get('term') for entry in ancestors if entry.get('term', '').lower() in disease_type_lower_set]

# df['parsed_ancestors'] = df['conditionBrowseModule_ancestors'].apply(parse_ancestors)

# df['selected_disease_names'] = df['parsed_ancestors'].apply(lambda x: extract_unique_sorted_terms(x, disease_type_set))

# df.drop(columns=['parsed_ancestors'], inplace=True)

# print(df[['conditionBrowseModule_ancestors', 'selected_disease_names']].head())
def extract_unique_sorted_terms(ancestors, disease_type_set):
    if isinstance(ancestors, np.ndarray):  # Convert NumPy arrays to Python lists
        ancestors = ancestors.tolist()

    if not isinstance(ancestors, list):  # Ensure it's a list
        print(f"Unexpected data format: {type(ancestors)} - {ancestors}")
        return []

    filtered_terms = {entry.get('term') for entry in ancestors if isinstance(entry, dict) and entry.get('term') in disease_type_set}

    if not filtered_terms:  # Log empty results for debugging
        print(f"No matches found in: {ancestors}")

    return sorted(filtered_terms)

# Apply function
df['selected_disease_names'] = df['conditionBrowseModule_ancestors'].apply(lambda x: extract_unique_sorted_terms(x, disease_type_set))

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(classes=disease_type)
dummies = pd.DataFrame(mlb.fit_transform(df['selected_disease_names']),
                       columns=mlb.classes_,
                       index=df.index)
dummies

In [ ]:
df = df.drop(columns=['selected_disease_names', 'conditionBrowseModule_ancestors'])

In [ ]:
df= pd.concat([df, dummies], axis=1)

In [ ]:
def extract_classes(collaborators_entry):
    """
    Extracts 'class' values from the collaborators entry.

    Parameters:
    - collaborators_entry (str, list, or np.ndarray): A list of dictionaries or an array.

    Returns:
    - list: A list of 'class' values. Returns an empty list if input is invalid or missing.
    """
    if isinstance(collaborators_entry, float) and pd.isna(collaborators_entry):  # Handle NaN values
        return []

    if isinstance(collaborators_entry, np.ndarray):  # Convert NumPy arrays to lists
        collaborators_entry = collaborators_entry.tolist()

    if isinstance(collaborators_entry, list):  # Ensure it's a list
        class_list = [entry.get('class') for entry in collaborators_entry if isinstance(entry, dict) and 'class' in entry]
        return class_list if class_list else []  # Ensure output is a list

    return []  # Default case for unexpected types

# Apply function with `.loc` to avoid SettingWithCopyWarning
df.loc[:, 'collaborator_classes'] = df['sponsorCollaboratorsModule_collaborators'].apply(extract_classes)

# Check the results
print(df['collaborator_classes'].head(10))

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

unique_classes = set(
    class_name
    for sublist in df['collaborator_classes'].dropna()
    for class_name in sublist
)

# Optional: Sort the classes for consistency
predefined_classes = sorted(unique_classes)

print("Unique Classes:", predefined_classes)

# Step 2: Replace NaN with empty lists
df['collaborator_classes'] = df['collaborator_classes'].apply(
    lambda x: x if isinstance(x, list) else []
)

# Step 3: Initialize MultiLabelBinarizer with predefined classes
mlb = MultiLabelBinarizer(classes=predefined_classes)

# Step 4: Fit and transform the 'collaborator_classes' column
dummies = pd.DataFrame(
    mlb.fit_transform(df['collaborator_classes']),
    columns=mlb.classes_,
    index=df.index
)

rename_dict = {col: f'collaborator_{col}' for col in dummies.columns}

# Rename the columns
dummies = dummies.rename(columns=rename_dict)

dummies

In [ ]:
df = pd.concat([df, dummies], axis=1)
df = df.drop(columns=['sponsorCollaboratorsModule_collaborators','collaborator_classes'])

Match the intervention names into broader Mesh terms

In [ ]:
def extract_d_ids(mesh_list):
    if isinstance(mesh_list, float) and pd.isna(mesh_list):  # Handle NaN values
        return []

    if isinstance(mesh_list, np.ndarray):  # Convert NumPy arrays to lists
        mesh_list = mesh_list.tolist()

    if isinstance(mesh_list, list):  # Process valid lists directly
        return [item['id'] for item in mesh_list if isinstance(item, dict) and 'id' in item and item['id'].startswith('D')]

    if isinstance(mesh_list, str):  # Convert string to a Python object
        try:
            parsed_list = ast.literal_eval(mesh_list)
            if isinstance(parsed_list, list):
                return [item['id'] for item in parsed_list if isinstance(item, dict) and 'id' in item and item['id'].startswith('D')]
        except (SyntaxError, ValueError):
            return []

    return []  # Default case for unexpected formats

# Apply function
df['interventions_mesh_ids'] = df['interventionBrowseModule_meshes'].apply(extract_d_ids)

In [ ]:
import requests

def get_mesh_tree_number(mesh_id):
    def fetch_tree_numbers(id_format):
        url = f"https://id.nlm.nih.gov/mesh/{id_format}.json"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            tree_numbers = data.get('treeNumber', [])
            if isinstance(tree_numbers, str):
                tree_numbers = [tree_numbers.split('/')[-1]]
            elif isinstance(tree_numbers, list):
                tree_numbers = [tn.split('/')[-1] for tn in tree_numbers]
            return tree_numbers
        return []

    # Try the original mesh_id first
    tree_numbers = fetch_tree_numbers(mesh_id)

    # If not found, try the shortened version (remove three zeros)
    if not tree_numbers and mesh_id.startswith('D') and len(mesh_id) == 10:
        shortened_id = mesh_id[:1] + mesh_id[4:]  # Remove extra zeros (D000017257 → D017257)
        tree_numbers = fetch_tree_numbers(shortened_id)

    # Extract only the first part of each tree number (e.g., D03, D10) and return unique values
    first_parts = {tn.split('.')[0] for tn in tree_numbers}
    return ', '.join(sorted(first_parts)) if first_parts else 'No tree number'

# Function to process each list of IDs in the column and get all tree numbers
def get_all_tree_numbers(id_list, row_index, total_rows):
    if not isinstance(id_list, list) or len(id_list) == 0:
        return 'No tree number'

    tree_parts = set()
    for mesh_id in id_list:
        tree_numbers = get_mesh_tree_number(mesh_id)
        if tree_numbers != 'No tree number':
            tree_parts.update(tree_numbers.split(', '))  # Split and collect individual tree numbers

    # Show progress after every 500 rows
    if (row_index + 1) % 500 == 0:
        print(f"Processed {row_index + 1}/{total_rows} rows...")

    return ', '.join(sorted(tree_parts)) if tree_parts else 'No tree number'

# Apply the function to the extracted_d_ids column
# df['tree_numbers'] = df['interventions_mesh_ids'].apply(get_all_tree_numbers)

total_rows = len(df)
df['tree_numbers'] = [get_all_tree_numbers(id_list, idx, total_rows) for idx, id_list in enumerate(df['interventions_mesh_ids'])]
# Display the updated DataFrame
df[['interventions_mesh_ids', 'tree_numbers']]

In [ ]:
df1 = df

In [ ]:
drugs_type = {"Inorganic Chemicals" : "D01",
              "Organic Chemicals": "D02",
              "Heterocyclic Compounds": "D03",
              "Polycyclic Compounds" : "D04",
              "Macromolecular Substances" : "D05",
              "Hormones, Hormone Substitutes, and Hormone Antagonists": "D06",
              "Enzymes and Coenzymes" : "D08",
              "Carbohydrates" : "D09",
              "Lipids" : "D10",
              "Amino Acids, Peptides, and Proteins" : "D12",
              "Nucleic Acids, Nucleotides, and Nucleosides" : "D13",
              "Complex Mixtures": "D20",
              "Biological Factors" : "D23",
              "Biomedical and Dental Materials" : "D25",
              "Pharmaceutical Preparations" : "D26",
              "Chemical Actions and Uses " : "D27"
}

code_to_drug = {v: k for k, v in drugs_type.items()}

# Function to extract relevant codes and create dummies
def extract_and_dummy(tree_numbers):
    codes = set(tree_numbers.split(", ")) if tree_numbers != "No tree number" else set()
    relevant_codes = codes.intersection(code_to_drug.keys())
    return {code_to_drug[code]: 1 for code in relevant_codes}

# Apply the function and create dummies
dummy_df = df["tree_numbers"].apply(extract_and_dummy).apply(pd.Series).fillna(0)
dummy_df.head(10)

In [ ]:
df = pd.concat([df, dummy_df], axis=1)

In [ ]:
num_missing = df['interventionBrowseModule_meshes'].isna().sum()
print(f"Number of missing rows: {num_missing}")
df = df.drop(columns = ['tree_numbers', 'interventions_mesh_ids','interventionBrowseModule_meshes'])

Merge trial descriptions and create embeddings

In [ ]:
def combine_descriptions(row):
    detailed_desc = row['descriptionModule_detailedDescription']
    brief_summary = row['descriptionModule_briefSummary']

    # Normalize whitespace in detailed_desc and split into words
    if pd.notnull(detailed_desc):
        normalized_desc = " ".join(detailed_desc.split())  # Removes extra spaces, newlines, and tabs
        word_count = len(normalized_desc.split())

        if word_count > 8:
            return f"{brief_summary} {normalized_desc}"  # Combine the two

    return brief_summary  # Return only the brief summary

# Create the new column with combined content
df['combined_description'] = df.apply(combine_descriptions, axis=1)

print("example combined description:", df['combined_description'][40])

In [ ]:
df = df.drop(columns = ['descriptionModule_detailedDescription', 'descriptionModule_briefSummary'])
average_word_count = df['combined_description'].apply(lambda x: len(str(x).split())).mean()
average_word_count

In [ ]:
df.to_parquet('/Users/jiazhengli/Desktop/project_root/completion_model.parquet', engine='pyarrow', compression='snappy')

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
from tqdm import tqdm  # For progress tracking

# Load PubMedBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")
model = AutoModel.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")

In [ ]:
# Move model to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

# Prepare to store embeddings as a PyTorch tensor
all_embeddings = []

# Process descriptions in batches
batch_size = 16  # Increase if you have sufficient GPU memory
num_batches = (len(df) + batch_size - 1) // batch_size  # Calculate number of batches

for i in tqdm(range(num_batches)):
    # Get the current batch of descriptions
    batch_descriptions = df['combined_description'][i * batch_size : (i + 1) * batch_size].tolist()

    # Tokenize and move tokens to GPU
    tokens = tokenizer(batch_descriptions, return_tensors='pt', truncation=True, max_length=256, padding='max_length')
    tokens = {key: val.to(device) for key, val in tokens.items()}

    # Get embeddings
    with torch.no_grad():
        outputs = model(**tokens)
        last_hidden_state = outputs.last_hidden_state  # Shape: (batch_size, sequence_length, hidden_size)
        cls_embeddings = last_hidden_state[:, 0, :]  # Extract CLS token (batch_size, 768)
        all_embeddings.append(cls_embeddings)

# Concatenate all batches into a single tensor and convert to NumPy
all_embeddings_flat = torch.cat(all_embeddings).cpu().numpy()  # Shape: (num_samples, 768)

# Ensure it matches the length of df
assert len(all_embeddings_flat) == len(df)

# Add the embeddings back to the DataFrame
df['cls_embedding'] = list(all_embeddings_flat)

Apply PCA dimension reduction

In [ ]:
from sklearn.decomposition import PCA

embeddings_matrix = np.vstack(df['cls_embedding'].values)

pca = PCA()
pca.fit(embeddings_matrix)

# Calculate cumulative explained variance
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)

# Plot cumulative explained variance
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance)
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.grid(True)
plt.axhline(y=0.95, color='r', linestyle='--', label='95% Variance')
plt.legend()
plt.show()

optimal_components = np.argmax(cumulative_variance >= 0.95) + 1
print(f"Optimal number of components: {optimal_components}")

In [ ]:
pca = PCA(n_components = optimal_components)
reduced_embeddings = pca.fit_transform(embeddings_matrix)

df['reduced_embedding'] = list(reduced_embeddings)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_parquet('/Users/jiazhengli/Desktop/trials_pmid_papers_results.parquet')
df

In [ ]:
df_all = pd.read_parquet('/Users/jiazhengli/Desktop/project_root/clinical_trials_combined.parquet')
df_all

In [ ]:
# df_all.columns[:50]
df_all['protocolSection.designModule.studyType']

In [ ]:
# # Make sure the column names are stripped of leading/trailing spaces
# df.columns = df.columns.str.strip()
# df_all.columns = df_all.columns.str.strip()

# # Perform the merge based on NCT ID
# df_merged = pd.merge(
#     df,
#     df_all[
#         [
#             'protocolSection.identificationModule.nctId',
#             'protocolSection.descriptionModule.briefSummary',
#             'protocolSection.descriptionModule.detailedDescription',
#             'protocolSection.identificationModule.briefTitle','protocolSection.identificationModule.officialTitle'
#         ]
#     ],
#     on='protocolSection.identificationModule.nctId',
#     how='left'
# )
# Make sure the column names are stripped of leading/trailing spaces
df.columns = df.columns.str.strip()
df_all.columns = df_all.columns.str.strip()

# Perform the merge based on NCT ID
df_merged = pd.merge(
    df,
    df_all[['protocolSection.identificationModule.nctId', 'protocolSection.armsInterventionsModule.interventions']],
    on='protocolSection.identificationModule.nctId',
    how='left'
)

In [ ]:
df_merged

In [ ]:
df_merged['protocolSection.descriptionModule.briefSummary'].isna().sum()

In [ ]:
df_merged['protocolSection.descriptionModule.detailedDescription'].isna().sum()

In [ ]:
import numpy as np

df_merged['intervention_types'] = df_merged['protocolSection.armsInterventionsModule.interventions'].apply(
    lambda x: [d.get('type') for d in x] if isinstance(x, (list, np.ndarray)) else None
)

In [ ]:
from collections import Counter
import pandas as pd

# Step 1: Drop nulls and convert each row to a set (to remove duplicates within a trial)
unique_types_per_trial = df_merged['intervention_types'].dropna().apply(lambda x: set(x))

# Step 2: Flatten the sets and count each unique type across all trials
flat_types = [t for types in unique_types_per_trial for t in types]
type_counts = Counter(flat_types)

# Step 3: Convert to DataFrame for display
type_counts_df = pd.DataFrame.from_dict(type_counts, orient='index', columns=['count']).reset_index()
type_counts_df.columns = ['intervention_type', 'count']


In [ ]:
type_counts_df

In [ ]:
df_merged.to_parquet('/Users/jiazhengli/Desktop/trials_pmid_papers_results.parquet')